In [289]:
# Import packages and read-in files

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os
import math 

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [3]:
# To use right xpress and get rid of unnecessary error codes
xp.init('C:/xpressmp/bin/xpauth.xpr')
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Formatting of all dataframes

In [290]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)

In [315]:
# Put all dataframes into 30-min slots instead of 5-min slots

# Ch A 30 min compressing
from datetime import time

# Resample to 30-minute intervals (use mean of groups)
ch_A_schedule_30min = ch_A_schedule.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 30)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_A_schedule_30min['time'] = ch_A_schedule_30min['Date'].dt.time
ch_A_schedule_30min = ch_A_schedule_30min[
    (ch_A_schedule_30min['time'] >= daily_start_time) &
    (ch_A_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_A_schedule_30min.reset_index(inplace = True)
ch_A_schedule_30min.drop('index', axis = 1,inplace = True)

# Ch 0 30 min compressing
from datetime import time
ch_0 = pd.DataFrame(ch_0_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_0_schedule_30min = ch_0.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_0_schedule_30min['time'] = ch_0_schedule_30min['Date'].dt.time
ch_0_schedule_30min = ch_0_schedule_30min[
    (ch_0_schedule_30min['time'] >= daily_start_time) &
    (ch_0_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_0_schedule_30min.reset_index(inplace = True)
ch_0_schedule_30min.drop('index', axis = 1,inplace = True)

# Ch 1 30 min compressing
from datetime import time
ch_1 = pd.DataFrame(ch_1_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_1_schedule_30min = ch_1.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_1_schedule_30min['time'] = ch_1_schedule_30min['Date'].dt.time
ch_1_schedule_30min = ch_1_schedule_30min[
    (ch_1_schedule_30min['time'] >= daily_start_time) &
    (ch_1_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_1_schedule_30min.reset_index(inplace = True)
ch_1_schedule_30min.drop('index', axis = 1,inplace = True)

# Ch 2 30 min compressing
from datetime import time
ch_2 = pd.DataFrame(ch_2_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_2_schedule_30min = ch_2.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_2_schedule_30min['time'] = ch_2_schedule_30min['Date'].dt.time
ch_2_schedule_30min = ch_2_schedule_30min[
    (ch_2_schedule_30min['time'] >= daily_start_time) &
    (ch_2_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])

ch_2_schedule_30min.reset_index(inplace = True)
ch_2_schedule_30min.drop('index', axis = 1,inplace = True)

# Getting Movie dataframe together

In [7]:
# Slot duration 30 minutes
slot_duration = 30  # minutes
movies_df['slots_needed'] = (movies_df['runtime_with_ads'] / slot_duration).apply(lambda x: int(x)).astype(int)

In [8]:
# Check for duplicate movie titles
duplicate_titles = movies_df[movies_df.duplicated(subset=['title'], keep=False)]
if not duplicate_titles.empty:
    print("Duplicate movie titles found:")
    print(duplicate_titles['title'])
else:
    print("No duplicate movie titles found.")

Duplicate movie titles found:
4                 The Avengers
17                     Titanic
76               The Lion King
105       Beauty and the Beast
149        Alice in Wonderland
                 ...          
5748              Midnight Sun
5761                The Island
5773            The Shaggy Dog
5855    Fun with Dick and Jane
5879        The Perfect Weapon
Name: title, Length: 258, dtype: object


## Choose Number of Days and Movies, then format time slots and datetime layout
Specify how many movies you want to practice code with and how many days you want schedule to be created for 

In [9]:
####
# Specify size of small_movies to practice and debug
movies_small = movies_df.head(550)

# make copy for movie info later since the model takes the movie titles out of movies_small
movies_copy = movies_small.copy(deep=True)

# change num_days based on number of days you want the scheduling function to create
num_days = 7

In [10]:
# create time slots and whatnot

from datetime import datetime, timedelta
# Define the broadcasting start and end dates
broadcast_start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
broadcast_end_date = broadcast_start_date + timedelta(days=num_days)  # 7 days including start date

# Define daily broadcast start and end times
daily_broadcast_start_time = timedelta(hours=7, minutes=0)
daily_broadcast_end_time = timedelta(hours=23, minutes=30)

# Generate all time slots over the date range
time_slots = []
current_date = broadcast_start_date
while current_date <= broadcast_end_date:
    # Set the start and end times for the current day
    day_start = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_start_time
    day_end = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_end_time
    
    current_time = day_start
    while current_time <= day_end:
        time_slots.append(current_time)
        current_time += timedelta(minutes=slot_duration)
    
    # Move to the next day
    current_date += timedelta(days=1)

# Create mappings between time slots and indices
time_to_index = {t: idx for idx, t in enumerate(time_slots)}
index_to_time = {idx: t for idx, t in enumerate(time_slots)}

# Function to calculate viewership of each movie (will be used in objective function)

In [11]:
# Function to find viewership of each movie based on popularity of each demographic
def get_views(m, t_idx, movies, our_channel):
    # Baseline viewership at time slot t_idx
    baseline = {
        'children': our_channel.iloc[t_idx]['children_baseline_view_count'],
        'adults': our_channel.iloc[t_idx]['adults_baseline_view_count'],
        'retirees': our_channel.iloc[t_idx]['retirees_baseline_view_count']
    }
    
    # Scaled popularity of movie m
    popularity = {
        'children': movies.loc[m, 'children_scaled_popularity'],
        'adults': movies.loc[m, 'adults_scaled_popularity'],
        'retirees': movies.loc[m, 'retirees_scaled_popularity']
    }
    
    # Calculate expected viewership (assuming total population of 1,000,000)
    total_population = 1_000_000
    viewership = sum(
        baseline[demo] * popularity[demo] * total_population
        for demo in ['children', 'adults', 'retirees']
    )
    
    return viewership

# The Model

In [12]:
# Function to create xpress problem, decision vars, and objective function
# to maximize viewership
def model(T, movies, our_channel):
    prob = xp.problem(name="Movie_Scheduling_Problem")
    
    # Constants
    slots_per_day = 34  # Number of slots per day (e.g., from 07:00 to 23:30)
    days = len(time_slots) // slots_per_day  # Number of days in the schedule
    M = len(time_slots)  # Big M for constraints
    T_end = len(time_slots) - 1  # Last time slot index

    
    # Decision Variables
    x = {(m, t): xp.var(vartype=xp.binary, name='x_{0}_{1}'.format(m, t)) 
         for m in movies.index for t in T}
    prob.addVariable(list(x.values()))
    
    y = {m: xp.var(vartype=xp.binary, name='y_{0}'.format(m)) for m in movies.index}
    prob.addVariable(list(y.values()))
 
    
    s = {m: xp.var(vartype=xp.integer, name='s_{0}'.format(m)) for m in movies.index}
    e = {m: xp.var(vartype=xp.integer, name='e_{0}'.format(m)) for m in movies.index}
    prob.addVariable(list(s.values()))
    prob.addVariable(list(e.values()))
    
    u = {(m, t): xp.var(vartype=xp.continuous, name='u_{0}_{1}'.format(m, t)) 
         for m in movies.index for t in T}
    prob.addVariable(list(u.values()))
    
    decision_vars = [x, y, s, e]

    # Constraints
    # 1. Movie Duration Constraint
    for m in movies.index:
        prob.addConstraint(
            xp.Sum(x[m, t] for t in T) == movies.loc[m, 'slots_needed'] * y[m]
        )
    # 2. Time Slot Occupancy Constraint
    for t in T:
        prob.addConstraint(
            xp.Sum(x[m, t] for m in movies.index) == 1
        )
    # 3. Start Time Constraints
    for m in movies.index:
        for t in T:
            prob.addConstraint(
                s[m] <= t * x[m, t] + (1 - x[m, t]) * M
            )
    # 4. End Time Constraints
    for m in movies.index:
        for t in T:
            prob.addConstraint(
                e[m] >= (t + 1) * x[m, t]
            )
    # 5. Movie Duration Relationship Constraint
    for m in movies.index:
        prob.addConstraint(
            e[m] - s[m] == movies.loc[m, 'slots_needed'] * y[m]
        )
    # 6 last movie ends at midnight
    prob.addConstraint(s[m] + movies.loc[m, 'slots_needed'] -1 <= T_end for m in movies.index)
    # 7. View Count Constraints
    for m in movies.index:
        for t_idx, t in enumerate(T):
            expected_viewership = get_views(m, t_idx, movies, our_channel)
            prob.addConstraint(
                u[m, t] == expected_viewership * x[m, t]
            )
    # 9. Objective Function
    prob.setObjective(
        xp.Sum(u[m, t] for m in movies.index for t in T),
        sense=xp.maximize
    )
    
    return prob, decision_vars


In [13]:
# Making schedule layout
def get_time(slot_index):
    return time_slots[slot_index].time()

used_movie_ids = []
def get_sched(prob, movies, decision_vars): 
    # target_genres = {'Animation', 'Fantasy', 'Adventure', 'Action', 'Science Fiction', 'Family'}
 
    x = decision_vars[0]
    y = decision_vars[1]
    s = decision_vars[2]
    e = decision_vars[3]
    
    scheduled_movies = []
    used_movie_ids = []

    for m_idx, m_row in movies.iterrows():
        y_value = prob.getSolution(y[m_idx])
        if y_value > 0.5:  # Movie is scheduled
            used_movie_ids.append(m_idx)
            start_slot = int(prob.getSolution(s[m_idx]))
            end_slot = int(prob.getSolution(e[m_idx]))

            # # Check if the movie is scheduled in the restricted time window (18:00 - 22:00)
            # if start_time >= datetime.strptime("18:00", "%H:%M").time() and start_time <= datetime.strptime("22:00", "%H:%M").time():
            #     movie_genre = movies.loc[m, 'genres']
                
            #     if isinstance(movie_genre, list):  # If genres are stored as a list
            #         if not any(genre in target_genres for genre in movie_genre):
            #             continue  # Skip this movie if no allowed genre is found
    
            # Ensure the end slot is valid
            if end_slot < len(time_slots):
                # Get the corresponding day and times
                start_time = time_slots[start_slot]
                end_time = time_slots[end_slot]
                day = start_time.strftime('%Y-%m-%d')
                start_time_formatted = start_time.strftime('%H:%M')
                end_time_formatted = end_time.strftime('%H:%M')
    
                # Append the movie schedule
                scheduled_movies.append({
                    'Time Slot': start_slot,
                    'Day': day,
                    'Start Time': start_time_formatted,
                    'End Time': end_time_formatted,
                    'Movie Index': m_idx,
                    'Movie Title': m_row['title'], 
                    'Genre': m_row['genres'],
                    'num_slots': m_row['slots_needed'],
                    'movie_budget': m_row['budget'],
                    'box_office_revenue': m_row['revenue'],
                    'n_ad_breaks': m_row['n_ad_breaks']
                })
    # Convert to a DataFrame for better organization
    schedule_df = pd.DataFrame(scheduled_movies) 
    
    # Sort by day and time slot
    schedule_df.sort_values(by=['Day', 'Time Slot'], inplace=True)
    
    # Reset index for cleaner output
    schedule_df.reset_index(drop=True, inplace=True)

    # Print the schedule
    print("Scheduled Movies in Order:")
    print(schedule_df)

    return scheduled_movies, used_movie_ids, schedule_df

In [14]:
# Putting it all together

def movie_sched(number_days, movies, our_channel):
    used_movie_ids= []
    total_schedule = [] # list of 
    scheduled_dates = {}  # Dictionary to track when movies were scheduled by movie_id
    
    full_list = []
    flat_list = []
    for k in range(number_days):
        if k == 0:
            T = range(k*34, ((k*34)+33)+1)
    
            prob, decision_vars =  model(T, movies, our_channel)
    
            prob.solve()
            
            scheduled_movies, used_movie_ids, schedule_df = get_sched(prob,movies,decision_vars)
            
            # add scheduled movies to a list
            full_list.append(scheduled_movies)
    
        # take used movies out of df so we don't repeat movies
            for i in used_movie_ids:
                movies.drop(i, inplace = True)
            total_schedule.append(scheduled_movies)

        elif k > 0:
            # 34*3 = 102 but index at zero --> 3 days is rows 0-101
            # 34*2 = 68 (start of day 3)
            # Day 1: 0-33  ( +33)
            # Day 2: 34-67 ( +34)
            # Day 3: 68-102 ( +34)
            
            T = range(k*34, (k+1)*34)
    
            prob, decision_vars =  model(T, movies, our_channel)
    
            prob.solve()
            
            scheduled_movies, used_movie_ids, schedule_df = get_sched(prob,movies,decision_vars)
            
            # add scheduled movies to a list
            full_list.append(scheduled_movies)
    
        # take used movies out of df so we don't repeat movies
            for i in used_movie_ids:
                movies.drop(i, inplace = True)
            total_schedule.append(scheduled_movies)
        
    # unpack the list of list of dictionaries so we can make it into a df
    for i in full_list:
        for item in i:
            flat_list.append(item)
            
    # make df of all movies used for all days, ordered by day then by time slot        
    full_df = pd.DataFrame.from_dict(flat_list, orient='columns')
    full_df.sort_values(by=['Day', 'Time Slot'], inplace=True)
    full_df.reset_index(drop=True, inplace=True)
        
    print('full df:', full_df)
    return full_df

In [15]:
# Code to find licensings fee (cost to air each movie)
import numpy as np
import pandas as pd
movie_money_info = pd.DataFrame()

def calculate_license_price(schedule_df: pd.DataFrame) -> pd.Series:
    '''
    Works out the cost required to buy a specific ad slot.  This is based on the time
    of day, and the budget/earnings of the movie being shown before the
    chosen ad slot.

    This function is applied to a schedule dataframe to create a new column
    containing the ad slot prices, returns NaN if the slot is not an ad slot.

    This is also multiplied by the prime time factor, desired profit margin does
    not take into account the effects of prime time factor currently, i.e.
    there'll be a larger profit margin obtained than the one specified for spots
    in prime time.

    Values used in generation of dataset.
    base_fee = 10_000
    profit_margin = 0.2
    budget_factor = 0.002
    box_office_revenue_factor = 0.001

    :param schedule_df: Dataframe containing the populated schedule with movies and
                      : ad breaks.
    :param base_fee: Base fee required for all movies to be licensed to a channel
    :param profit_margin: Percent (in 0-1 scale) of license fee that the channel
                        : wants to make in profit.
    :param budget_factor: What percent (in 0-1 scale) of the movie's budget contributes
                        : to the license fee.
    :param box_office_factor: What percent (in 0-1 scale) of the movie's box office renvenue
                            : contributes to the license fee.
    '''

    # license_fee = (base_fee
    #                + (budget_factor * schedule_df.movie_budget)
    #                + (box_office_factor *schedule_df.box_office_revenue)
    #                ) * (1. + profit_margin)

    license_fee = (10_000
                   + (0.002 * schedule_df.movie_budget)
                   + (0.001 *schedule_df.box_office_revenue)
                   ) * (1. + 0.2)

    ad_slot_cost = (license_fee / schedule_df.n_ad_breaks) #* schedule_df.prime_time_factor
    
    movie_money_info['movie'] = schedule_df['Movie Title'] 
    movie_money_info['license_fee'] = round(license_fee,2)
    movie_money_info['cost_per_ad_slot_cost'] = round(ad_slot_cost,2)
    movie_money_info['num_slots'] = schedule_df['n_ad_breaks']

    # return np.round(ad_slot_cost, 2)
    return movie_money_info

# Find money information based on generated schedule (change num_days and size of movies_small if desired, but make sure you run all cells again)
## df returned from license_fee function is called "movie_money_info" and can be referenced later on in code 

In [16]:
# test for 3 days 
# 
seven_days = movie_sched(number_days = num_days, movies = movies_small, our_channel= ch_A_schedule_30min)

FICO Xpress v9.4.2, Hyper, solve started 12:52:18, Nov 25, 2024
Heap usage: 29MB (peak 29MB, 9179KB system)
Maximizing MILP Movie_Scheduling_Problem using up to 12 threads and up to 7528MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     57784 rows        39050 cols       152350 elements     20350 entities
Presolved problem has:
     38534 rows        20350 cols       114400 elements     20350 entities
LP relaxation tightened
Presolve finished in 1 seconds
Heap usage: 43MB (peak 68MB, 9179KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.31e-02,  1.71e+05] / [ 3.91e-03,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.72e+02] / [ 1.00e+00,  2.72e+02]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 3.31e-02,  1.71e+05]
Autoscaling applied standard scaling

Symmetric problem: generators: 24, support set: 1295
 Numb

In [17]:
seven_days

,Time Slot,Day,Start Time,End Time,Movie Index,Movie Title,Genre,num_slots,movie_budget,box_office_revenue,n_ad_breaks
0,0,2024-10-01,07:00,09:30,539,Thor: Love and Thunder,"['Fantasy', 'Action', 'Comedy']",5,250000000,760928081,4
1,5,2024-10-01,09:30,13:30,17,Titanic,"['Drama', 'Romance']",8,200000000,2264162353,7
2,13,2024-10-01,13:30,16:30,10,Forrest Gump,"['Comedy', 'Drama', 'Romance']",6,55000000,677387716,5
3,19,2024-10-01,16:30,18:30,93,The Hangover,['Comedy'],4,35000000,469310836,4
4,23,2024-10-01,18:30,22:00,22,The Wolf of Wall Street,"['Crime', 'Drama', 'Comedy']",7,100000000,392000000,7
5,30,2024-10-01,22:00,07:00,403,After,"['Romance', 'Drama']",4,14000000,69497587,4
6,34,2024-10-02,07:00,09:30,48,Harry Potter and the Deathly Hallows: Part 2,"['Fantasy', 'Adventure']",5,125000000,1341511219,5
7,39,2024-10-02,09:30,12:30,192,GoodFellas,"['Drama', 'Crime']",6,25000000,46835000,5
8,45,2024-10-02,12:30,15:30,14,The Shawshank Redemption,"['Drama', 'Crime']",6,25000000,28341469,5
9,51,2024-10-02,15:30,18:00,358,Cruella,"['Comedy', 'Crime']",5,200000000,233503234,5


In [340]:
seven_lic = calculate_license_price(schedule_df= seven_days)
seven_lic

,movie,license_fee,cost_per_ad_slot_cost,num_slots
0,Thor: Love and Thunder,1525113.70,381278.42,4
1,Titanic,3208994.82,458427.83,7
2,Forrest Gump,956865.26,191373.05,5
3,The Hangover,659173.00,164793.25,4
4,The Wolf of Wall Street,722400.00,103200.00,7
5,After,128997.10,32249.28,4
6,Harry Potter and the Deathly Hallows: Part 2,1921813.46,384362.69,5
7,GoodFellas,128202.00,25640.40,5
8,The Shawshank Redemption,106009.76,21201.95,5
9,Cruella,772203.88,154440.78,5


In [19]:

def fill_our_channel_slots(df_of_movies):
    # create our movie df based on number of days we schedule
    our_channel = ch_A_schedule_30min.head(34*num_days)
    # our_channel.drop(['children_baseline_view_count', 'adults_baseline_view_count','retirees_baseline_view_count'], inplace=True)
    # fill our channel schedule dataframe with info
    d = 0
    x = 0
    p = []
    p_flat = []
    # for movie titles and ads in the right slots
    for i in df_of_movies['num_slots']:
        # val = df_of_movies['Movie Title'].iloc[d]
        our_channel.loc[x:x+i, 'movie'] = df_of_movies['Movie Title'].iloc[d]
        our_channel.loc[x:x+i, 'ad_cost'] = movie_money_info['cost_per_ad_slot_cost'].iloc[d]
        d += 1
        x += i
    # for calculating the expected viewcounts and costs per viewer
    for m in our_channel['movie']:
        v = movies_copy.index[movies_copy['title'] == m].to_list()
        p.append(v)
    for i in p:
        for it in i:
            p_flat.append(it)
    
    child_expected_viewcounts = our_channel ['children_baseline_view_count']*movies_copy.loc[it, 'children_scaled_popularity']
    adults_expected_viewcounts = our_channel ['adults_baseline_view_count']*movies_copy.loc[it, 'adults_scaled_popularity']
    retirees_expected_viewcounts = our_channel ['retirees_baseline_view_count']*movies_copy.loc[it, 'retirees_scaled_popularity']
    
    our_channel ['child_expected_viewcounts'] = child_expected_viewcounts
    our_channel ['adults_expected_viewcounts'] = adults_expected_viewcounts
    our_channel ['retirees_expected_viewcounts'] = retirees_expected_viewcounts
    our_channel ['total_expected_viewers'] = (our_channel ['child_expected_viewcounts']+our_channel ['adults_expected_viewcounts']+our_channel ['retirees_expected_viewcounts']*1000000)
    our_channel ['cost_per_viewer'] = our_channel ['ad_cost']/our_channel ['total_expected_viewers']

    our_channel.drop(['children_baseline_view_count','adults_baseline_view_count','retirees_baseline_view_count','prime_time_factor'],axis =1, inplace = True)
    return our_channel

In [213]:
our_channel = fill_our_channel_slots(df_of_movies = seven_days)
our_channel

,Date,movie,ad_cost,child_expected_viewcounts,adults_expected_viewcounts,retirees_expected_viewcounts,total_expected_viewers,cost_per_viewer
0,2024-10-01 07:00:00,Thor: Love and Thunder,381278.42,0.011464,0.021978,0.002136,2135.822766,178.515945
1,2024-10-01 07:30:00,Thor: Love and Thunder,381278.42,0.009158,0.022601,0.002776,2776.392565,137.328714
2,2024-10-01 08:00:00,Thor: Love and Thunder,381278.42,0.005727,0.023899,0.003561,3561.213456,107.064186
3,2024-10-01 08:30:00,Thor: Love and Thunder,381278.42,0.002806,0.025864,0.004502,4501.974832,84.691371
4,2024-10-01 09:00:00,Thor: Love and Thunder,381278.42,0.001083,0.028480,0.005606,5605.978259,68.012825
...,...,...,...,...,...,...,...,...
233,2024-10-07 21:30:00,Real Steel,127024.44,0.001198,0.104823,0.008552,8551.910738,14.853340
234,2024-10-07 22:00:00,Real Steel,127024.44,0.000647,0.099854,0.007101,7101.343072,17.887383
235,2024-10-07 22:30:00,Real Steel,127024.44,0.000328,0.094353,0.005806,5806.096378,21.877770
236,2024-10-07 23:00:00,Real Steel,127024.44,0.000156,0.088441,0.004674,4674.583467,27.173424


In [21]:
def channel_comparisons(our_channel_df):
    # Create a dataframe with aggreagte information 

    all_slots = pd.DataFrame()
    # cost per ad slot on each channel
    all_slots['0'] = ch_0_schedule_30min['ad_slot_price']
    all_slots['1'] = ch_1_schedule_30min['ad_slot_price']
    all_slots['2'] = ch_2_schedule_30min['ad_slot_price']
    
    # basic stats
    # all_slots['avg'] = round(all_slots.mean(axis = 1), 2)
    all_slots['max'] = all_slots.max(axis=1)
    all_slots['min'] = all_slots.min(axis=1)
    all_slots['our price'] = round(our_channel_df['ad_cost'],2)
    
    # identify best and worst channel 
    all_slots['max ch #'] = all_slots[['0','1','2']].idxmax(axis=1)
    all_slots['min ch #'] = all_slots[['0','1','2']].idxmin(axis=1)
    
    # find out percentage difference of most expensive ad compared to ours
    all_slots['times_more_expensive'] = round(all_slots['our price']/all_slots['max'],2)

    return all_slots.head(34*num_days)

In [22]:
channel_comparisons(our_channel_df = our_channel)

,0,1,2,max,min,our price,max ch #,min ch #,times_more_expensive
0,56995.17,25458.66,365283.35,365283.35,25458.66,381278.42,2,1,1.04
1,56995.17,25458.66,365283.35,365283.35,25458.66,381278.42,2,1,1.04
2,25479.06,16629.10,365283.35,365283.35,16629.10,381278.42,2,1,1.04
3,25479.06,16629.10,365283.35,365283.35,16629.10,381278.42,2,1,1.04
4,25479.06,16629.10,17700.00,25479.06,16629.10,381278.42,0,1,14.96
...,...,...,...,...,...,...,...,...,...
233,36992.15,52292.17,135239.84,135239.84,36992.15,127024.44,2,0,0.94
234,24661.43,27792.85,NaN,27792.85,24661.43,127024.44,1,0,4.57
235,24661.43,27792.85,63672.56,63672.56,24661.43,127024.44,2,0,1.99
236,24661.43,27792.85,63672.56,63672.56,24661.43,127024.44,2,0,1.99


##
buy and sell ads
sell- viewership loses
buy- viewership gains
buy- always increasing revenue 

# Calculate Ad Slot Conversion Rates

In [31]:
# create list of genres
genres = set({})
for movie in movies_small['genres']:
    # print(movie)
    movie_list = eval(movie)
    # print(type(movie_list))
    for genre in movie_list:
        genres.add(genre)
print(genres)

# string = "['Action', 'Science Fiction', 'Adventure']"
# list_from_string = eval(string)
# print(list_from_string)


{'Action', 'Fantasy', 'Crime', 'Drama', 'Adventure', 'Western', 'Animation', 'Family', 'Horror', 'Mystery', 'Music', 'History', 'War', 'Science Fiction', 'Romance', 'Comedy', 'Thriller'}


In [75]:
# get rid of "Date" column in conversion rate matrices (only care about the 30-min slots)
ch_0_conversion_rates.reset_index(inplace = True)
ch_0_conversion_rates.drop(['Date'], axis=1 ,inplace = True)

ch_1_conversion_rates.reset_index(inplace = True)
ch_1_conversion_rates.drop(['Date'], axis=1 ,inplace = True)

ch_2_conversion_rates.reset_index(inplace = True)
ch_2_conversion_rates.drop(['Date'], axis=1 ,inplace = True)

In [79]:
ch_1_conversion_rates.head(5)

,Western,Adventure,Crime,Thriller,Fantasy,Documentary,Music,Action,TV Movie,Drama,Mystery,Science Fiction,Comedy,Family,Horror,War,Animation,History,Romance
0,0.0,0.0,0.202073,0.0,0.0,0.0,0.0,0.0,0.0,0.202073,0.202073,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.101036,0.0,0.0,0.0,0.0,0.0,0.0,0.224780,0.101036,0.0,0.123744,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.247487,0.000000,0.0,0.247487,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.247487,0.000000,0.0,0.247487,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.247487,0.000000,0.0,0.247487,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
ch_0_conversion_rates.drop(['Date'], axis=1 ,inplace = True)
ch_0_conversion_rates.head()

,Western,Adventure,Crime,Thriller,Fantasy,Documentary,Music,Action,TV Movie,Drama,Mystery,Science Fiction,Comedy,Family,Horror,War,Animation,History,Romance
0,0.0,0.0,0.0,0.1750,0.0,0.0,0.0,0.0,0.0,0.1750,0.1750,0.0,0.000000,0.0,0.1750,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0875,0.0,0.0,0.0,0.0,0.0,0.0875,0.0875,0.0,0.123744,0.0,0.0875,0.0,0.0,0.0,0.123744
2,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.247487,0.0,0.0000,0.0,0.0,0.0,0.247487
3,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.247487,0.0,0.0000,0.0,0.0,0.0,0.247487
4,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.247487,0.0,0.0000,0.0,0.0,0.0,0.247487


In [205]:
our_channel.head(10)

,Date,movie,ad_cost,child_expected_viewcounts,adults_expected_viewcounts,retirees_expected_viewcounts,total_expected_viewers,cost_per_viewer
0,2024-10-01 07:00:00,Thor: Love and Thunder,381278.42,0.011464,0.021978,0.002136,2135.822766,178.515945
1,2024-10-01 07:30:00,Thor: Love and Thunder,381278.42,0.009158,0.022601,0.002776,2776.392565,137.328714
2,2024-10-01 08:00:00,Thor: Love and Thunder,381278.42,0.005727,0.023899,0.003561,3561.213456,107.064186
3,2024-10-01 08:30:00,Thor: Love and Thunder,381278.42,0.002806,0.025864,0.004502,4501.974832,84.691371
4,2024-10-01 09:00:00,Thor: Love and Thunder,381278.42,0.001083,0.028480,0.005606,5605.978259,68.012825
5,2024-10-01 09:30:00,Titanic,458427.83,0.000347,0.031728,0.006874,6874.275087,66.687443
6,2024-10-01 10:00:00,Titanic,458427.83,0.000136,0.035577,0.008300,8299.931974,55.232721
7,2024-10-01 10:30:00,Titanic,458427.83,0.000151,0.039992,0.009867,9866.616581,46.462516
8,2024-10-01 11:00:00,Titanic,458427.83,0.000293,0.044922,0.011548,11547.706962,39.698603
9,2024-10-01 11:30:00,Titanic,458427.83,0.000580,0.050310,0.013306,13306.113476,34.452421


In [301]:
our_channel['total_expected_viewers'].sum(axis = 0)

3204352.9108007266

In [214]:
# find which of our genres are used in this week's schedule
gen_list = []
movie_gen = []
for i in seven_days['Genre']:
    gen_list.append(i)
# print(gen_list)

for el in gen_list:
    j = eval(el)  # makes each one a list
    movie_gen.append(j)
print(movie_gen)


    # for l in j:
    #     flat.append(l) # make list of the lists
    # print(flat)
# unis = list(set(flat)) # make list into one set, each genre in seven_days listed once
# print(unis)
        


[['Fantasy', 'Action', 'Comedy'], ['Drama', 'Romance'], ['Comedy', 'Drama', 'Romance'], ['Comedy'], ['Crime', 'Drama', 'Comedy'], ['Romance', 'Drama'], ['Fantasy', 'Adventure'], ['Drama', 'Crime'], ['Drama', 'Crime'], ['Comedy', 'Crime'], ['Action', 'Drama'], ['Drama', 'Crime'], ['Action', 'Adventure', 'Comedy'], ['Drama', 'Romance', 'Thriller'], ['Drama', 'Romance'], ['Drama', 'Romance'], ['Drama', 'Action', 'Crime', 'Thriller'], ['Horror', 'Mystery', 'Crime'], ['Crime', 'Mystery', 'Thriller'], ['Adventure', 'Fantasy'], ['Drama', 'History'], ['Fantasy', 'Drama', 'Crime'], ['Adventure', 'Fantasy', 'Action'], ['Thriller', 'Action', 'Crime'], ['Action', 'Drama', 'Thriller'], ['Action', 'Adventure', 'Fantasy'], ['Drama'], ['Drama', 'Action', 'History'], ['Adventure', 'Fantasy'], ['Fantasy', 'Action', 'Adventure'], ['Adventure', 'Drama', 'Science Fiction'], ['Adventure', 'Fantasy'], ['Drama', 'Crime'], ['Comedy'], ['Adventure', 'Fantasy'], ['Action', 'Thriller', 'Crime'], ['Science Fiction

In [228]:
print(len(our_channel), len(movie_gen), len(seven_days['num_slots']))

238 43 43


In [229]:
seven_days.head()

,Time Slot,Day,Start Time,End Time,Movie Index,Movie Title,Genre,num_slots,movie_budget,box_office_revenue,n_ad_breaks
0,0,2024-10-01,07:00,09:30,539,Thor: Love and Thunder,"['Fantasy', 'Action', 'Comedy']",5,250000000,760928081,4
1,5,2024-10-01,09:30,13:30,17,Titanic,"['Drama', 'Romance']",8,200000000,2264162353,7
2,13,2024-10-01,13:30,16:30,10,Forrest Gump,"['Comedy', 'Drama', 'Romance']",6,55000000,677387716,5
3,19,2024-10-01,16:30,18:30,93,The Hangover,['Comedy'],4,35000000,469310836,4
4,23,2024-10-01,18:30,22:00,22,The Wolf of Wall Street,"['Crime', 'Drama', 'Comedy']",7,100000000,392000000,7


In [243]:
# add genre column to our channel
gen_df = pd.DataFrame()
gen_df['movie'] = seven_days['Movie Title']
gen_df['genre'] = ''

for i in range(len(gen_df)):
    gen_df.at[i, 'genre'] = movie_gen[i]


gen_df.head()

,movie,genre
0,Thor: Love and Thunder,"[Fantasy, Action, Comedy]"
1,Titanic,"[Drama, Romance]"
2,Forrest Gump,"[Comedy, Drama, Romance]"
3,The Hangover,[Comedy]
4,The Wolf of Wall Street,"[Crime, Drama, Comedy]"


In [ ]:
# ratio of (ad cost)/(conversion_rate*1_000_000)



In [30]:
# our_channel.head()

In [46]:
# all_genres = ['Action', 'Fantasy', 'Crime', 'Drama', 'Adventure', 'Western', 'Animation', 'Family', 'Horror', 'Mystery', 'Music', 'History', 'War', 'Science Fiction', 'Romance', 'Comedy', 'Thriller']
# gen_list = []
# for g in all_genres:
#     for i in range(3):
#         gen = '{}{}'.format(g,i)
#         gen_list.append(gen)


# all_conv_rates = pd.DataFrame(columns = gen_list)
# all_conv_rates

In [256]:
genres = ['Western','Adventure','Crime','Thriller',	'Fantasy', 'Documentary','Music','Action','TV Movie','Drama','Mystery','Science Fiction','Comedy','Family',	'Horror','War','Animation',	'History','Romance']

ch_0_conversion_rates['max'] = ch_0_conversion_rates.max(axis=1)
ch_0_conversion_rates['min'] = ch_0_conversion_rates.min(axis=1)

ch_0_conversion_rates['max gen'] = ch_0_conversion_rates[genres].idxmax(axis = 1)
ch_0_conversion_rates['min gen'] = ch_0_conversion_rates[genres].idxmin(axis = 1)




# all_slots = pd.DataFrame()
#     # cost per ad slot on each channel
#     all_slots['0'] = ch_0_schedule_30min['ad_slot_price']
#     all_slots['1'] = ch_1_schedule_30min['ad_slot_price']
#     all_slots['2'] = ch_2_schedule_30min['ad_slot_price']
    
#     # basic stats
#     # all_slots['avg'] = round(all_slots.mean(axis = 1), 2)
#     all_slots['max'] = all_slots.max(axis=1)
#     all_slots['min'] = all_slots.min(axis=1)
#     all_slots['our price'] = round(our_channel_df['ad_cost'],2)
    
#     # identify best and worst channel 
#     all_slots['max ch #'] = all_slots[['0','1','2']].idxmax(axis=1)
#     all_slots['min ch #'] = all_slots[['0','1','2']].idxmin(axis=1)
    
#     # find out percentage difference of most expensive ad compared to ours
#     all_slots['times_more_expensive'] = round(all_slots['our price']/all_slots['max'],2)

In [300]:
# ch_0_conversion_rates.reset_index(inplace = True)
# ch_0_conversion_rates.head(34*7)

In [273]:
ch_1_conversion_rates.reset_index(inplace = True)
ch_2_conversion_rates.reset_index(inplace = True)


In [275]:
Thrillers = pd.DataFrame()
Thrillers['0'] = ch_0_conversion_rates['Thriller']
Thrillers['1'] = ch_1_conversion_rates['Thriller']
Thrillers['2'] = ch_2_conversion_rates['Thriller']

Thrillers.head(34*7)

,0,1,2
0,0.1750,0.0000,0.0
1,0.0875,0.0000,0.0
2,0.0000,0.0000,0.0
3,0.0000,0.0000,0.0
4,0.0000,0.0000,0.0
...,...,...,...
233,0.0000,0.0875,0.0
234,0.0000,0.1750,0.0
235,0.0000,0.1750,0.0
236,0.0000,0.1750,0.0


In [271]:
ch_1_conversion_rates['Thriller']

Date
2024-10-01 07:05:00    0.000000
2024-10-01 07:35:00    0.000000
2024-10-01 08:10:00    0.000000
2024-10-01 08:40:00    0.000000
2024-10-01 09:10:00    0.000000
                         ...   
2024-12-23 21:45:00    0.247487
2024-12-23 22:15:00    0.123744
2024-12-23 22:50:00    0.000000
2024-12-23 23:20:00    0.000000
2024-12-23 23:50:00    0.000000
Name: Thriller, Length: 2676, dtype: float64

In [284]:
zero_sum = round(ch_0['ad_slot_price'].sum(),2)
one_sum = round(ch_1['ad_slot_price'].sum(),2)
two_sum = round(ch_2['ad_slot_price'].sum(),2)
print(zero_sum, one_sum, two_sum)

203877775.33 199209934.51 203932191.88


In [287]:
print(max(zero_sum, one_sum, two_sum), min(zero_sum, one_sum, two_sum))

203932191.88 199209934.51


In [302]:
our_channel['total_expected_viewers'].sum(axis = 0)

3204352.9108007266

In [293]:
act0 = ch_0_conversion_rates['Action'].mean(axis = 0)
drama0 = ch_0_conversion_rates['Drama'].mean(axis = 0)
fant0 = ch_0_conversion_rates['Fantasy'].mean(axis = 0)
print(act0, drama0, fant0)

0.06323408891414514 0.09235057559063808 0.024598159619596632


In [306]:
tot0 = ((act0*drama0*fant0)*1_000_000)*100

In [307]:
act1 = ch_1_conversion_rates['Action'].mean(axis = 0)
drama1 = ch_1_conversion_rates['Drama'].mean(axis = 0)
fant1 = ch_1_conversion_rates['Fantasy'].mean(axis = 0)

tot1 = ((act1*drama1*fant1)*1_000_000)*100

In [309]:
act2 = ch_2_conversion_rates['Action'].mean(axis = 0)
drama2 = ch_2_conversion_rates['Drama'].mean(axis = 0)
fant2 = ch_2_conversion_rates['Fantasy'].mean(axis = 0)

tot2 = ((act2*drama2*fant2)*1_000_000)*100

In [311]:
(tot0+tot1+tot2)*10

400335.0116321766

# assume lose 21% of viewership every week (--> 210,000 people)

- ratio = budget/viewership gain
- total week license fee: 42,176,877.15
- total week license fee * 0.67 = 28,258,507.69
- total week license fee * 0.74 = 31,210,889.09

# buy 80 ads per channel per week:
- budget: 17,783,380.76
- viewership gain: 400,335
- ratio: 44.42124910387551

# buy 60 ads/ch/wk:
- budget: 8,900,160.64
- viewership gain: 320,268
- ratio: 27.78972810271398

# buy 40 ads/ch/wk:
- budget: 7,190,515.86
- viewership gain: 240,201
- ratio: 29.93541184258184

# buy 20 ads/ch/wk:
- budget: 3,864,272.82
- viewership gain: 160,134
- ratio: 24.131494997939225




In [343]:
(seven_lic['license_fee'].sum())*0.74

31210889.091

In [328]:
ch_0.dropna(inplace = True)
ch0 = ch_0.sample()
zero = ch0['ad_slot_price'].sum()

ch_1.dropna(inplace = True)
ch1 = ch_1.sample(60)
one = ch1['ad_slot_price'].sum()

ch_2.dropna(inplace = True)
ch2 = ch_2.sample(60)
two = ch2['ad_slot_price'].sum()

budget_opt_60 = one+zero+two
budget_opt_60

8900160.64

In [330]:
ch_0.dropna(inplace = True)
ch0 = ch_0.sample(80)
zero = ch0['ad_slot_price'].sum()

ch_1.dropna(inplace = True)
ch1 = ch_1.sample(80)
one = ch1['ad_slot_price'].sum()

ch_2.dropna(inplace = True)
ch2 = ch_2.sample(80)
two = ch2['ad_slot_price'].sum()

budget_opt_80 = one+zero+two
budget_opt_80

17783380.76

In [334]:
ch_0.dropna(inplace = True)
ch0 = ch_0.sample(40)
zero = ch0['ad_slot_price'].sum()

ch_1.dropna(inplace = True)
ch1 = ch_1.sample(40)
one = ch1['ad_slot_price'].sum()

ch_2.dropna(inplace = True)
ch2 = ch_2.sample(40)
two = ch2['ad_slot_price'].sum()

budget_opt_40 = one+zero+two
budget_opt_40

7190515.86

In [332]:
ch_0.dropna(inplace = True)
ch0 = ch_0.sample(20)
zero = ch0['ad_slot_price'].sum()

ch_1.dropna(inplace = True)
ch1 = ch_1.sample(20)
one = ch1['ad_slot_price'].sum()

ch_2.dropna(inplace = True)
ch2 = ch_2.sample(20)
two = ch2['ad_slot_price'].sum()

budget_opt_20 = one+zero+two
budget_opt_20

3864272.8199999994